In [1]:
!pip install streamlit langchain openai yfinance matplotlib python-dotenv

In [ ]:
# app.py

import os
import streamlit as st
from dotenv import load_dotenv
import openai
import yfinance as yf
import matplotlib.pyplot as plt
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType

# 🌱 환경변수 로드
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    st.error("❌ OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")
    st.stop()

openai.api_key = api_key

# 🧠 LangChain Chat Model 설정
llm = ChatOpenAI(temperature=0, openai_api_key=api_key)

# 📈 주가 분석 도구 정의
def get_stock_data(ticker: str, period="6mo"):
    stock = yf.Ticker(ticker)
    hist = stock.history(period=period)
    return hist

def analyze_stock(ticker: str):
    hist = get_stock_data(ticker)
    description = f"""
    종목 코드: {ticker}
    기간: 최근 6개월
    평균 종가: {hist['Close'].mean():.2f}
    최고가: {hist['High'].max():.2f}
    최저가: {hist['Low'].min():.2f}
    """
    return description

# LangChain Tool로 등록
tools = [
    Tool(
        name="Stock Data Analyzer",
        func=analyze_stock,
        description="주어진 종목 코드에 대해 주가 분석 정보를 제공하는 함수입니다. 예: 'AAPL'"
    )
]

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# 🖥️ Streamlit 인터페이스
st.title("📊 LangChain 기반 주가 분석기")

ticker = st.text_input("🔍 종목 코드를 입력하세요 (예: AAPL, MSFT, TSLA 등)", value="AAPL")

if st.button("분석 시작"):
    with st.spinner("데이터 분석 중..."):
        # 데이터 시각화
        hist = get_stock_data(ticker)
        st.subheader(f"{ticker} 주가 차트 (최근 6개월)")
        st.line_chart(hist['Close'])

        # LangChain 에이전트 사용
        query = f"{ticker}의 최근 주가 분석을 해줘"
        response = agent.run(query)

        st.subheader("🧠 LangChain 분석 결과")
        st.write(response)
